In [16]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as f
import matplotlib.pyplot as plt

In [17]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
num_epochs = 4
batch_size =4
learning_rate =0.001

In [19]:
# dataset has pil image of range [0,1]
# we tranform them to tensors of normalized range [-1,1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [20]:
train_dataset = torchvision.datasets.CIFAR10(root = './',train=True,download=True,transform=transform)

Files already downloaded and verified


In [21]:
test_dataset = torchvision.datasets.CIFAR10(root = './',train=False,download=True,transform=transform)

Files already downloaded and verified


In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

TypeError: __init__() got an unexpected keyword argument 'lr'